In [9]:
%%capture
pip install cobra

# General Add reaction methods

In [10]:
#create a Reaction 
#reaction = Reaction('R_3OAS140')
#reaction.name = '3 oxoacyl acyl carrier protein synthase n C140 '
#reaction.subsystem = 'Cell Envelope Biosynthesis'
#reaction.lower_bound = 0.  # This is the default
#reaction.upper_bound = 1000.  # This is the default

In [11]:
#Create metabolites
#ACP_c = Metabolite(
    #'ACP_c',
    #formula='C11H21N2O7PRS',
    #name='acyl-carrier-protein',
    #compartment='c')

In [12]:
#ADD metabolites to reactions
#reaction.add_metabolites({
    #malACP_c: -1.0,
    #h_c: -1.0,
    #ddcaACP_c: -1.0,
    #co2_c: 1.0,
    #ACP_c: 1.0,
    #omrsACP_c: 1.0
#})

In [13]:
#Add Gene reaction rules
#reaction.gene_reaction_rule = '( STM2378 or STM1197 )'
#reaction.genes

In [14]:
#Add the reaction to the model

#model.add_reactions([reaction])#

# **Preparation and data import** 


In [15]:
import cobra

In [16]:
Model_path = "/content/drive/MyDrive/Halomonas_Model_devt/Halomonas_elongata_Model.xml"
#Model_path = "/content/drive/MyDrive/Halomonas_Model_devt/HalomonasmodelV1.xml"

In [17]:
from cobra import Model, Reaction, Metabolite

In [18]:
model = cobra.io.read_sbml_model(Model_path)

In [19]:
model

Name,Halomonas_M9
Memory address,0x07f62214eda50
Number of metabolites,1524
Number of reactions,2296
Number of groups,0
Objective expression,1.0*Growth - 1.0*Growth_reverse_699ae
Compartments,"cytosol, extracellular space, periplasm"


In [20]:
#model.summary()
model.get_metabolite_compartments()

{'C_c', 'C_e', 'C_p'}

#Inlude the pathway for PHB synthesis, not included in the draft model. 
There wwere no formulas in the BIGG databse for phb and hydroxybutyrl-CoA so i had to obtain it from the PubCHEM. 

Inorder to confirm these pathways i will run a visualisation in Escher in jupyter lab to see how connected the whole thing is. 

##Add  reaction PhbC

In [21]:


##Necessary Metabolites
#Create Phb metabolite
phb_c = Metabolite(
    'phb_c',
    formula='C4H8O3',
    name='Poly-beta-hydroxybutyrate',
    compartment='C_c')

# create metabolite 3hbcoa_R_c (written as h3bcoa for convenience )
h3bcoa__R_c =  Metabolite(
    '3hbcoa__R_c',
    formula='C25H42N7O18P3S',
    name='(R)-3-Hydroxybutyryl-CoA',
    compartment='C_c')

#get coa_c
coa_c = model.metabolites.get_by_id('coa_c')


#create a phb synthesis(Polymerization) Reaction 
PhbC_reaction = Reaction('PHBS_syn_1')
PhbC_reaction.name = 'PHB synthetase '
PhbC_reaction.subsystem = 'Butanote metabolism'
PhbC_reaction.lower_bound = -1000.0  # This is the default
PhbC_reaction.upper_bound = 1000.0  # This is the default


#ADD metabolites to PhC reactions 
PhbC_reaction.add_metabolites({
    h3bcoa__R_c: -1.0,
    coa_c: 1.0,
    phb_c: 1.0
})


#Gene reaction rule from RAST
#PhbC_reaction.gene_reaction_rule = 'fig|6666666.858611.peg.1520'
#PhbC_reaction.genes

##Add reaction Phb Sink

In [22]:
##DONT INCLUDE THIS

#Final metabolite (Phb) introduced as a Sink reaction because it is an intracellular polymer that accumulates in cell
#cytosolic PHB
#Add Phb sink reactions 
#phb_reaction = Reaction('sink_phb_c')
#phb_reaction.name = 'PHB sink transport reaction'
#phb_reaction.subsystem = 'Butanote metabolism'
#phb_reaction.lower_bound = 0.0  # This is the default check other snk reactions to know their bounds 
#phb_reaction.upper_bound = 1000.0  # This is the default

#ADD metabolites to Sink reactions
#phb_reaction.add_metabolites({
#    phb_c: -1.0
#})

#Gene reaction rule from Sink 
#phb_reaction.gene_reaction_rule = 'Spontaneous'
#phb_reaction.genes


##Add reaction PhbB

In [23]:
# add reaction for Acetoacetyl CoA reductase PhbB
#initialize metabolites
h_c = model.metabolites.get_by_id('h_c')
nadph_c = model.metabolites.get_by_id('nadph_c')
nadp_c = model.metabolites.get_by_id('nadp_c')
aacoa_c = model.metabolites.get_by_id('aacoa_c')

#Add PhbB reactions 
PhbB_reaction = Reaction('AACOAR_syn')
PhbB_reaction.name = 'Acetoacetyl CoA reductase'
PhbB_reaction.subsystem = 'Butanote metabolism'
PhbB_reaction.lower_bound = -1000.0  # This is the default check other snk reactions to know their bounds 
PhbB_reaction.upper_bound = 1000.0  # This is the default
#ADD metabolites to PhB reactions PhbC
PhbB_reaction.add_metabolites({
    h3bcoa__R_c: 1.0,
    aacoa_c: -1.0,
    h_c: -1.0,
    nadp_c: 1.0,
    nadph_c: -1.0
})
#Add Gene reaction rules
#PhbB_reaction.gene_reaction_rule = '(fig|6666666.858611.peg.8 or fig|6666666.858611.peg.841)'
#PhbB_reaction.genes


In [24]:
model

Name,Halomonas_M9
Memory address,0x07f62214eda50
Number of metabolites,1524
Number of reactions,2296
Number of groups,0
Objective expression,1.0*Growth - 1.0*Growth_reverse_699ae
Compartments,"cytosol, extracellular space, periplasm"


##Add reaction PhaA

In [25]:
#PhaA reaction
PhbA_reaction = Reaction('KAT1')
PhbA_reaction.name = '3-ketoacyl-CoA thiolase'
PhbA_reaction.subsystem = 'Butanote metabolism'
PhbA_reaction.lower_bound = -1000.0  # This is the default check other snk reactions to know their bounds 
PhbA_reaction.upper_bound = 1000.0  # This is the default

#ADD metabolites to PhA reactions
PhbA_reaction.add_metabolites({
    aacoa_c: -1.0,
    aacoa_c: 2.0,
    coa_c: 1.0,
    })

#Add Gene reaction rules
#PhbA_reaction.gene_reaction_rule = '(fig|6666666.858611.peg.1593 or fig|6666666.858611.peg.1761)'
#PhbA_reaction.genes

##Add reaction Enoyl-CoA hydratase

In [26]:
#Enzyme name in BIGG database as 3-hydroxyacyl-CoA dehydratase
#so i be using this reaction considering the components are same 




##Adding all reactions to the model

In [27]:
#putting all the reactions in the model
model.add_reaction(PhbC_reaction)
model.add_reaction(PhbB_reaction)
model.add_reaction(PhbA_reaction)#already existed in the model reason for commenting it out
#model.add_reaction(Sink_reaction)
#The above line or this one
#strictly for testing. Want to see if the reaction stays a strict sink reaction or used as above, the difference that can ensue
#model.add_boundary(phb_c, type = 'sink', lb = 0.0, ub = 1000)

Ignoring reaction 'KAT1' since it already exists.


In [28]:
model.sinks

[]

In [29]:
model.reactions.EX_glc__D_e.lower_bound = -10.0
model.optimize()

,fluxes,reduced_costs
14GLUCANabcpp,0.000000,-5.551115e-17
14GLUCANtexi,0.000000,0.000000e+00
1PPDCRc,0.000000,-5.313598e-02
2AGPE120tipp,0.000000,0.000000e+00
2AGPE140tipp,0.000000,0.000000e+00
...,...,...
ATPM,0.000000,-8.174767e-03
4HBZR,0.000139,-1.110223e-16
ASNS1,0.000000,-2.452430e-02
PHBS_syn_1,0.000000,0.000000e+00


In [30]:
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
asn__L_e,EX_asn__L_e,9,4,37.50%
ca2_e,EX_ca2_e,0.007257,0,0.00%
cl_e,EX_cl_e,0.007257,0,0.00%
cobalt2_e,EX_cobalt2_e,0.0001394,0,0.00%
cu2_e,EX_cu2_e,0.0009886,0,0.00%
fe3_e,EX_fe3_e,6.957,0,0.00%
glc__D_e,EX_glc__D_e,10,6,62.50%
k_e,EX_k_e,0.2722,0,0.00%
mg2_e,EX_mg2_e,0.0121,0,0.00%
mn2_e,EX_mn2_e,0.0009635,0,0.00%


In [31]:
model.objective = 'PHBS_syn_1' #maximum possible PHB production under such glucose assimilation conditions
model.summary()


Metabolite,Reaction,Flux,C-Number,C-Flux
Metabolite,Reaction,Flux,C-Number,C-Flux


In [32]:
model.objective = 'Growth'
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
asn__L_e,EX_asn__L_e,9,4,37.50%
ca2_e,EX_ca2_e,0.007257,0,0.00%
cl_e,EX_cl_e,0.007257,0,0.00%
cobalt2_e,EX_cobalt2_e,0.0001394,0,0.00%
cu2_e,EX_cu2_e,0.0009886,0,0.00%
fe3_e,EX_fe3_e,6.957,0,0.00%
glc__D_e,EX_glc__D_e,10,6,62.50%
k_e,EX_k_e,0.2722,0,0.00%
mg2_e,EX_mg2_e,0.0121,0,0.00%
mn2_e,EX_mn2_e,0.0009635,0,0.00%


In [33]:
from cobra.flux_analysis import flux_variability_analysis
flux_variability_analysis(model) #report these results for FVA using Glucose

,minimum,maximum
14GLUCANabcpp,0.000000,0.000000e+00
14GLUCANtexi,0.000000,0.000000e+00
1PPDCRc,0.000000,1.104217e-11
2AGPE120tipp,0.000000,1.740142e-11
2AGPE140tipp,0.000000,1.628050e-11
...,...,...
ATPM,0.000000,1.418906e-10
4HBZR,0.000139,1.394311e-04
ASNS1,0.000000,4.729688e-11
PHBS_syn_1,0.000000,0.000000e+00


In [34]:
cobra.io.write_sbml_model(model, "/content/drive/MyDrive/Halomonas_Model_devt/Halomonas_elongata_ModelV2.xml")
#cobra.io.save_json_model(model, "/content/drive/MyDrive/Model_devt_Ch/FinalModel_V2.json")

In [35]:
%%capture
model.boundary

#Test the Pathway

In [36]:
model.metabolites.get_by_id('3hbcoa__R_c')

Metabolite identifier,3hbcoa__R_c
Name,(R)-3-Hydroxybutyryl-CoA
Memory address,0x07f62206f6710
Formula,C25H42N7O18P3S
Compartment,C_c
In 2 reaction(s),"PHBS_syn_1, AACOAR_syn"


In [37]:
#change the carbon source to see how it affects the roduction of PHB by seeing the changes in flux variablity.
medium = model.medium
medium['EX_fru_e'] = 10.0
model.medium = medium
model.reactions.EX_glc__D_e.lower_bound = 0.0
model.reactions.EX_fru_e.lower_badound = -10.0
#model.reactions.EX_nh4_e.lower_bound = 1000.0

In [38]:
model.medium

{'EX_asn__L_e': 9.0,
 'EX_ca2_e': 1000.0,
 'EX_cl_e': 1000.0,
 'EX_cobalt2_e': 1000.0,
 'EX_cu2_e': 1000.0,
 'EX_fe3_e': 1000.0,
 'EX_fru_e': 10.0,
 'EX_h2o_e': 1000.0,
 'EX_k_e': 1000.0,
 'EX_mg2_e': 1000.0,
 'EX_mn2_e': 1000.0,
 'EX_o2_e': 1000.0,
 'EX_pi_e': 1000.0,
 'EX_so4_e': 1000.0,
 'EX_zn2_e': 1000.0}

In [39]:
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
asn__L_e,EX_asn__L_e,9,4,37.50%
ca2_e,EX_ca2_e,0.007257,0,0.00%
cl_e,EX_cl_e,0.007257,0,0.00%
cobalt2_e,EX_cobalt2_e,0.0001394,0,0.00%
cu2_e,EX_cu2_e,0.0009886,0,0.00%
fe3_e,EX_fe3_e,6.957,0,0.00%
fru_e,EX_fru_e,10,6,62.50%
k_e,EX_k_e,0.2722,0,0.00%
mg2_e,EX_mg2_e,0.0121,0,0.00%
mn2_e,EX_mn2_e,0.0009635,0,0.00%


In [40]:
flux_variability_analysis(model)

,minimum,maximum
14GLUCANabcpp,0.000000,0.000000e+00
14GLUCANtexi,0.000000,0.000000e+00
1PPDCRc,0.000000,-3.664644e-12
2AGPE120tipp,0.000000,-7.047531e-12
2AGPE140tipp,0.000000,-1.189271e-11
...,...,...
ATPM,0.000000,1.470180e-10
4HBZR,0.000139,1.394311e-04
ASNS1,0.000000,4.900598e-11
PHBS_syn_1,0.000000,0.000000e+00


In [41]:
#was no phb produced in glucose only, nitrogen limiting conditions, fructose and  glucose, fructose alone. No phb was produced, we make sure tomorrow that the connectivity of the model has 
#not been broken with the reaction, maybe it doesnt connect with other components of the model. use escher to do this
#Try to add the other Fatty acid reactions. FInd if they are in the model already(my guess is there is)
#see how they all connect

In [42]:
cobra.flux_analysis.find_blocked_reactions(model)

['14GLUCANabcpp',
 '14GLUCANtexi',
 '2DHGLCK',
 '2HH24DDH',
 '2HH24DDH1',
 '2HXMPt6',
 '2PGLYCt6',
 '34DHCINMtex',
 '34DHCINMtpp',
 '34dhpactex',
 '3HOXPACt2pp',
 '3hoxpactex',
 '3HPAOX',
 '3MBt2pp',
 '3MBt4pp',
 '3MBtex',
 '42A12BOOXpp',
 '4CMCOAS',
 '4H2KPILY',
 '4HOXPACMOF',
 '4HOXPACt2pp',
 'AAMYL',
 'ABUTtex',
 'ACGApts',
 'ACGAptspp',
 'ACGAtex',
 'ACMANApts',
 'ACMANAptspp',
 'ACMANAtex',
 'ACMUMptspp',
 'ACMUMtex',
 'ACTD',
 'ACTD2',
 'ACTDa',
 'ADNt2',
 'ALAGLUE',
 'ALCD3',
 'ALCD4',
 'ALDD1',
 'ALDD19xr',
 'ALDD31',
 'ALDD31_1',
 'ALDD4',
 'ALDD6',
 'AMMQT8_2',
 'AMPTASEPG',
 'ARABDI',
 'ARAI',
 'ASO3t8pp',
 'ASO3tex',
 'ASO4rt2rpp',
 'ASO4tex',
 'ASR',
 'BETALDHx',
 'BETALDHy',
 'BHMT',
 'BMOCOS',
 'BMOGDS1',
 'BMOGDS2',
 'BTDD_RR',
 'BTOHt2rpp',
 'BTOHtex',
 'BUTCT',
 'BZt1pp',
 'BZtex',
 'CACOAHA',
 'CAFFCOA',
 'CHOLD',
 'CHOLD2',
 'CHOLSabc',
 'CHOLSabc_1',
 'CHOLS_ex',
 'CHOLSH',
 'CHTBSptspp',
 'CHTBStex',
 'CMCMSAD',
 'CMHMI',
 'COCOAHA',
 'CPMPS',
 'CYNTAH',
 'CYNTt2'

In [43]:
model.reactions.PHBS_syn_1

Reaction identifier,PHBS_syn_1
Name,PHB synthetase
Memory address,0x07f62206f6590
Stoichiometry,3hbcoa__R_c <=> coa_c + phb_c (R)-3-Hydroxybutyryl-CoA <=> Coenzyme A + Poly-beta-hydroxybutyrate
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [44]:
model.reactions.AACOAR_syn

Reaction identifier,AACOAR_syn
Name,Acetoacetyl CoA reductase
Memory address,0x07f62206e9b10
Stoichiometry,aacoa_c + h_c + nadph_c <=> 3hbcoa__R_c + nadp_c Acetoacetyl-CoA + H+ + Nicotinamide adenine dinucleotide phosphate - reduced <=> (R)-3-Hydroxybutyryl-CoA + Nicotinamide adenine dinucleotide phosphate
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [45]:
model.reactions.KAT1.lower_bound = -1000.0
model.reactions.KAT1

Reaction identifier,KAT1
Name,3-ketoacyl-CoA thiolase
Memory address,0x07f622025add0
Stoichiometry,aacoa_c + coa_c <=> 2.0 accoa_c Acetoacetyl-CoA + Coenzyme A <=> 2.0 Acetyl-CoA
GPR,fig_6666666_879901_peg_1830 or fig_6666666_879901_peg_2360
Lower bound,-1000.0
Upper bound,1000.0


In [46]:
model.metabolites.aacoa_c

Metabolite identifier,aacoa_c
Name,Acetoacetyl-CoA
Memory address,0x07f62211fe610
Formula,C25H36N7O18P3S
Compartment,C_c
In 9 reaction(s),"HMGCOASi, ACACCT, KAT1, OCOAT1, FAO4, AACOAR_syn, HACD1, ACACT1r, HACD1i"


In [47]:
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
asn__L_e,EX_asn__L_e,9,4,37.50%
ca2_e,EX_ca2_e,0.007257,0,0.00%
cl_e,EX_cl_e,0.007257,0,0.00%
cobalt2_e,EX_cobalt2_e,0.0001394,0,0.00%
cu2_e,EX_cu2_e,0.0009886,0,0.00%
fe3_e,EX_fe3_e,6.957,0,0.00%
fru_e,EX_fru_e,10,6,62.50%
k_e,EX_k_e,0.2722,0,0.00%
mg2_e,EX_mg2_e,0.0121,0,0.00%
mn2_e,EX_mn2_e,0.0009635,0,0.00%


In [48]:
consistent_model = cobra.flux_analysis.fastcc(model)

In [49]:
model.objective = "Growth"
consistent_model.optimize()

,fluxes,reduced_costs
1PPDCRc,0.000000,-5.313598e-02
2AGPE120tipp,0.000000,5.551115e-17
2AGPE140tipp,0.000000,0.000000e+00
2AGPE141tipp,0.000000,-5.551115e-17
2AGPE160tipp,0.000000,-2.043692e-03
...,...,...
sink_sheme_c,0.000000,0.000000e+00
Growth,1.439781,1.387779e-16
ATPM,0.000000,-8.174767e-03
4HBZR,0.000139,-6.461064e-13


In [50]:
model.reactions.EX_bz_e.lower_bound = -0.0
medium = model.medium
medium["EX_bz_e"] = 9.7e-05
model.medium = medium
model.medium

{'EX_asn__L_e': 9.0,
 'EX_bz_e': 9.7e-05,
 'EX_ca2_e': 1000.0,
 'EX_cl_e': 1000.0,
 'EX_cobalt2_e': 1000.0,
 'EX_cu2_e': 1000.0,
 'EX_fe3_e': 1000.0,
 'EX_fru_e': 10.0,
 'EX_h2o_e': 1000.0,
 'EX_k_e': 1000.0,
 'EX_mg2_e': 1000.0,
 'EX_mn2_e': 1000.0,
 'EX_o2_e': 1000.0,
 'EX_pi_e': 1000.0,
 'EX_so4_e': 1000.0,
 'EX_zn2_e': 1000.0}

In [51]:
model.optimize()

,fluxes,reduced_costs
14GLUCANabcpp,0.000000,0.000000e+00
14GLUCANtexi,0.000000,0.000000e+00
1PPDCRc,0.000000,-5.313598e-02
2AGPE120tipp,0.000000,0.000000e+00
2AGPE140tipp,0.000000,0.000000e+00
...,...,...
ATPM,0.000000,-8.174767e-03
4HBZR,0.000042,-1.110223e-16
ASNS1,0.000000,-2.452430e-02
PHBS_syn_1,0.000000,-0.000000e+00


In [52]:
model.add_boundary(model.metabolites.get_by_id("phb_c"), type="demand")

Reaction identifier,DM_phb_c
Name,Poly-beta-hydroxybutyrate demand
Memory address,0x07f621eb1f950
Stoichiometry,phb_c --> Poly-beta-hydroxybutyrate -->
GPR,
Lower bound,0
Upper bound,1000.0


In [53]:
model.optimize()

,fluxes,reduced_costs
14GLUCANabcpp,0.000000,-2.775558e-17
14GLUCANtexi,0.000000,0.000000e+00
1PPDCRc,0.000000,-5.313598e-02
2AGPE120tipp,0.000000,0.000000e+00
2AGPE140tipp,0.000000,0.000000e+00
...,...,...
4HBZR,0.000042,0.000000e+00
ASNS1,0.000000,-2.452430e-02
PHBS_syn_1,0.000000,-0.000000e+00
AACOAR_syn,0.000000,-0.000000e+00


In [54]:
model.objective = "PHBS_syn_1"
model.optimize()

,fluxes,reduced_costs
14GLUCANabcpp,0.000000,-6.661338e-16
14GLUCANtexi,0.000000,0.000000e+00
1PPDCRc,0.000000,-5.000000e-01
2AGPE120tipp,0.000000,-1.083333e-01
2AGPE140tipp,0.000000,-1.333333e-01
...,...,...
4HBZR,0.000000,-1.333333e-01
ASNS1,0.000000,-2.000000e-01
PHBS_syn_1,17.841773,0.000000e+00
AACOAR_syn,17.841773,0.000000e+00


In [55]:
model.objective = "Growth"
model.optimize()

,fluxes,reduced_costs
14GLUCANabcpp,0.000000,-2.775558e-17
14GLUCANtexi,0.000000,0.000000e+00
1PPDCRc,0.000000,-5.313598e-02
2AGPE120tipp,0.000000,-1.328400e-02
2AGPE140tipp,0.000000,-1.634953e-02
...,...,...
4HBZR,0.000042,0.000000e+00
ASNS1,0.000000,-2.452430e-02
PHBS_syn_1,0.000000,0.000000e+00
AACOAR_syn,0.000000,1.110223e-16


In [56]:
#use this notebook to fix things around here. solves the polymer production problem. demand plus compartment corection.
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
asn__L_e,EX_asn__L_e,9,4,37.50%
bz_e,EX_bz_e,9.7E-05,7,0.00%
ca2_e,EX_ca2_e,0.007257,0,0.00%
cl_e,EX_cl_e,0.007257,0,0.00%
cobalt2_e,EX_cobalt2_e,0.0001394,0,0.00%
cu2_e,EX_cu2_e,0.0009886,0,0.00%
fe3_e,EX_fe3_e,6.957,0,0.00%
fru_e,EX_fru_e,10,6,62.50%
k_e,EX_k_e,0.2722,0,0.00%
mg2_e,EX_mg2_e,0.0121,0,0.00%


In [61]:
cobra.io.write_sbml_model(model, "/content/drive/MyDrive/Halomonas_Model_devt/Halomonas_elongata_ModelV2.xml")
#cobra.io.save_json_model(model, "/content/drive/MyDrive/Model_devt_Ch/FinalModel_V2.json")

In [58]:
model

Name,Halomonas_M9
Memory address,0x07f62214eda50
Number of metabolites,1526
Number of reactions,2299
Number of groups,0
Objective expression,1.0*Growth - 1.0*Growth_reverse_699ae
Compartments,"cytosol, extracellular space, periplasm"


In [59]:
model.metabolites.phb_c

Metabolite identifier,phb_c
Name,Poly-beta-hydroxybutyrate
Memory address,0x07f62206f6650
Formula,C4H8O3
Compartment,C_c
In 2 reaction(s),"PHBS_syn_1, DM_phb_c"


In [60]:
model.get_metabolite_compartments()

{'C_c', 'C_e', 'C_p'}